In [4]:
import pandas as pd
from finvizfinance.quote import finvizfinance
from skollama.models.ollama.classification.zero_shot import ZeroShotOllamaClassifier
from skollama.models.ollama.classification.few_shot import FewShotOllamaClassifier

In [16]:
pd.set_option('display.max_colwidth', None)  # No column width limit

### Pull stock news headline data

In [ ]:
# Initialize the finvizfinance object for Apple
stock = finvizfinance('AAPL')

# Fetch the latest news articles
news_df = stock.ticker_news()

news_df.head()

,Date,Title,Link,Source
0,2024-11-15 15:30:00,AST SpaceMobile Could Revolutionize Cellphone ...,https://finance.yahoo.com/m/14945860-72f3-3215...,Investor's Business Daily
1,2024-11-15 14:37:00,Apple Inc. (AAPL): Among the Best Metaverse St...,https://finance.yahoo.com/news/apple-inc-aapl-...,Insider Monkey
2,2024-11-15 14:13:00,"Apple, Gen Z on personal finance: Wealth",https://finance.yahoo.com/video/apple-gen-z-pe...,Yahoo Finance Video
3,2024-11-15 13:53:00,Apple Removes Russian-Language RFE/RL App Foll...,https://finance.yahoo.com/news/apple-removes-r...,GuruFocus.com
4,2024-11-15 13:38:00,"Stocks fall, growth in e-commerce: Morning Brief",https://finance.yahoo.com/video/stocks-fall-gr...,Yahoo Finance Video


In [ ]:
# Reorder Columns
news_df = news_df[['Date','Link','Title']]

# Define the keywords to filter by
keywords = ['AAPL', 'Apple']

pattern = '|'.join(keywords)

# Filter the DataFrame  
filtered_news_df = news_df[news_df['Title'].str.contains(pattern, case=False, na=False)]

filtered_news_df.head()

,Date,Link,Title
1,2024-11-15 14:37:00,https://finance.yahoo.com/news/apple-inc-aapl-...,Apple Inc. (AAPL): Among the Best Metaverse St...
2,2024-11-15 14:13:00,https://finance.yahoo.com/video/apple-gen-z-pe...,"Apple, Gen Z on personal finance: Wealth"
3,2024-11-15 13:53:00,https://finance.yahoo.com/news/apple-removes-r...,Apple Removes Russian-Language RFE/RL App Foll...
5,2024-11-15 12:43:00,https://finance.yahoo.com/m/8fe66b95-b994-3fc7...,Trump Promises Tariffs. Wall Street Is Debatin...
7,2024-11-15 12:02:00,https://finance.yahoo.com/news/berkshire-hatha...,"Berkshire Hathaway Sells $23B in Apple Shares,..."


### Run Zero Shot Classifier

In [12]:
# Initialize the ZeroShotOllamaClassifier
clf = ZeroShotOllamaClassifier(model='llama3')

# Define the candidate labels
candidate_labels = ['positive', 'negative', 'neutral']

# Fit the classifier (no training data needed for zero-shot)
clf.fit(None, candidate_labels)

# Predict the sentiment of each news title as a new colum in our DataFrame
filtered_news_df['Sentiment_zero'] = clf.predict(filtered_news_df['Title'])

100%|██████████| 59/59 [05:20<00:00,  5.42s/it]
C:\Users\karti\AppData\Local\Temp\ipykernel_10656\3810096421.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_news_df['Sentiment_zero'] = clf.predict(filtered_news_df['Title'])


In [17]:
filtered_news_df[['Title','Sentiment_zero']].head()

,Title,Sentiment_zero
1,Apple Inc. (AAPL): Among the Best Metaverse Stocks To Buy According to Hedge Funds,positive
2,"Apple, Gen Z on personal finance: Wealth",positive
3,Apple Removes Russian-Language RFE/RL App Following Pressure from Russian Authorities,negative
5,Trump Promises Tariffs. Wall Street Is Debating What That Means for Apple.,neutral
7,"Berkshire Hathaway Sells $23B in Apple Shares, Adjusts Portfolio in Q3 2024",neutral


### Train and Run Few Shot Classifier
Start by randomly selecting a few training examples from the original dataset

In [ ]:
# Randomly select headlines for few-shot training and add a training indicator
few_shot_df = filtered_news_df.sample(n=7, random_state=1)
filtered_news_df['Few Shot Training Example'] = filtered_news_df.index.isin(few_shot_df.index)

# View training examples
list(few_shot_df['Title'])

C:\Users\karti\AppData\Local\Temp\ipykernel_10656\2926192509.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_news_df['Few Shot Training Example'] = filtered_news_df.index.isin(few_shot_df.index)


['Jim Cramer on Apple Inc. (AAPL)s Operations in China: China And Trump, They Are Also Like Oil And Water And If You Do Business There, Its Gonna Hurt You Here',
 'Apple Removes Russian-Language RFE/RL App Following Pressure from Russian Authorities',
 'Is Apple Inc. (AAPL) Best Virtual Reality Stock To Buy Now?',
 'Apple to Launch Smart Home Camera in 2026, Expand AirPods Health Features',
 'Apple making foray into smart home IP camera market, says analyst Ming-Chi Kuo',
 'Apple Stock and These 4 Are Top Picks to Own in 2025',
 'Where Will Apple Stock Be in 5 Years?']

In [ ]:
# Manually assigned labels corresponding to the selected headlines

user_labels = [
    'negative',
    'negative',
    'neutral',
    'positive',
    'positive',
    'positive',
    'neutral'
]

# Add the user-provided labels to the few-shot DataFrame
few_shot_df['User_Sentiment'] = user_labels

### Initialise and run few shot classifier on the rest of the dataset

In [ ]:
# Initialize the FewShotOllamaClassifier
few_shot_clf = FewShotOllamaClassifier(model='llama3')

# Fit the classifier with user-provided examples directly from the DataFrame columns
few_shot_clf.fit(few_shot_df['Title'], few_shot_df['User_Sentiment'])

# Predict the sentiment of all news titles in the filtered DataFrame
filtered_news_df['Sentiment_few'] = few_shot_clf.predict(filtered_news_df['Title'])

In [ ]:
few_shots_filtered_news_df = filtered_news_df[['Title','Sentiment_zero','Sentiment_few','Few Shot Training Example']]
few_shots_filtered_news_df